***Participantes (RM - NOME):***<br>
Vinicius - 345167<br>
Guilherme - 346006<br>
Fabiana - 345858<br>

###**Criar um classificador de chamados aplicando técnicas de PLN**
---

A **DinDinAgora** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

Obs.: Para a métrica F1 Score, usar o parâmetro average = 'weighted'.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

Bom desenvolvimento!

In [22]:
# Google Colab install dependencies
!pip install -U sklearn
!pip install -U nltk
!pip install -U pandas

  Using cached sklearn-0.0.post1-py3-none-any.whl


In [23]:
# Download dataset in colab
!mkdir data
!mkdir models
!wget -O data/tickets_reclamacoes_classificados.csv https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘models’: File exists
--2023-03-04 17:00:48--  https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv
Resolving dados-ml-pln.s3.sa-east-1.amazonaws.com (dados-ml-pln.s3.sa-east-1.amazonaws.com)... 16.12.0.22
Connecting to dados-ml-pln.s3.sa-east-1.amazonaws.com (dados-ml-pln.s3.sa-east-1.amazonaws.com)|16.12.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33020135 (31M) [text/csv]
Saving to: ‘data/tickets_reclamacoes_classificados.csv’

data/tickets_reclam 100%[===================>]  31.49M  34.2MB/s    in 0.9s    

2023-03-04 17:00:49 (34.2 MB/s) - ‘data/tickets_reclamacoes_classificados.csv’ saved [33020135/33020135]



In [24]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
import pickle
import nltk
from nltk.stem.rslp import RSLPStemmer
from nltk.tokenize import word_tokenize
nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to /home/vinimarcili/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/vinimarcili/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vinimarcili/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
df = pd.read_csv('data/tickets_reclamacoes_classificados.csv', delimiter=';')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


In [27]:
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


In [28]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = RSLPStemmer()

# Stemming
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

# Remoção de stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Tokenização
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = remove_stopwords(tokens)
    stems = stem_tokens(tokens, stemmer)
    return stems

# Criação do modelo de vetorização para pipe final, usando unigramas
vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 1), min_df=1, max_df=1.0, use_idf=True, smooth_idf=True, sublinear_tf=True)
vectorizer.fit(df['descricao_reclamacao'])
pickle.dump(vectorizer, open('models/vectorizer.pkl', 'wb'))

/home/vinimarcili/anaconda3/envs/ln/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [29]:
# Separando em treino de teste depois da vetorização e stemização
X = vectorizer.transform(df['descricao_reclamacao'])
y = df['categoria']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [30]:

# Regressão Logistica
model = LogisticRegression(random_state=42, max_iter=3000)
model.fit(X_train, y_train)
pickle.dump(model, open('models/model_logistic.pkl', 'wb'))
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.8684364828125639

In [31]:
# Arvore de decisão
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
pickle.dump(clf, open('models/model_tree.pkl', 'wb'))
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.7352073180917162

In [32]:
# Multinominal Naive Bayes
multi = MultinomialNB()
multi.fit(X_train, y_train)
pickle.dump(multi, open('models/model_multi.pkl', 'wb'))
y_pred = multi.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.6718314148008369

In [33]:
# Random Forest
random = RandomForestClassifier(random_state=42)
random.fit(X_train, y_train)
pickle.dump(random, open('models/model_random.pkl', 'wb'))
y_pred = random.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.780892985183201

In [34]:
# Linear SVC
linear = LinearSVC()
linear.fit(X_train, y_train)
pickle.dump(linear, open('models/model_linear.pkl', 'wb'))
y_pred = linear.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.8579976698497366

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

In [35]:
# Caregando dataframe
data_final = pd.read_csv('data/tickets_reclamacoes_classificados.csv', delimiter=';')

In [36]:
# Stopwords e Instancia do Stemmer
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = RSLPStemmer()

In [37]:
# Função de stemming
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [38]:
# Remoção de stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

In [39]:
# Tokenização
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = remove_stopwords(tokens)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [40]:
# Criação do modelo de vetorizaçã para pipe final, usando unigramas
vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 1), min_df=1, max_df=1.0, use_idf=True, smooth_idf=True, sublinear_tf=True)
vectorizer.fit(df['descricao_reclamacao'])
pickle.dump(vectorizer, open('models/vectorizer.pkl', 'wb'))

/home/vinimarcili/anaconda3/envs/ln/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [41]:
# Separando em treino de teste depois da vetorização e stemização
X = vectorizer.transform(df['descricao_reclamacao'])
y = df['categoria']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [42]:
# Regressão Logistica
model = LogisticRegression(random_state=42, max_iter=3000)
model.fit(X_train, y_train)
pickle.dump(model, open('models/model_logistic.pkl', 'wb'))
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.8684364828125639

In [43]:
# teste
model.predict(vectorizer.transform(['Ola, meu nome é João e estou com problemas com o meu cartão de crédito']))

array(['Cartão de crédito / Cartão pré-pago'], dtype=object)